In [ ]:
"This Jupyter Notebook is a copy Databricks notebook and was used in the batch data processing part of this project"

In [ ]:
from pyspark.sql.functions import *
import urllib

In [ ]:
"""
    Why was this code used?
    
    This code was used to define the path to the delta tables and read the delta tables into the Spark dataframe.
"""
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [ ]:
"""
    Why was this code used?
    
    This code was used to Get the AWS access key and secret key from the spark dataframe and Encode the secrete key.
"""
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
"""
    Why was this code used?
    
    This code was used to identify the AWS S3 bucket name ,Mount name for the bucket,Source url,Mount the drive
"""
AWS_S3_BUCKET = "user-0abf7f0cd605-bucket"
MOUNT_NAME = "/mnt/0abf7f0cd605_mount/"
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

In [ ]:
display(dbutils.fs.ls("/0abf7f0cd605_mount../.."))

In [ ]:
%sql

SET spark.databricks.delta.formatCheck.enabled=false

In [ ]:
"""
    Why was this code used?
    
    This code was used to identify File location and type , 
    sterisk(*) indicates reading all the content of the specified file that have .json extension , 
    Ask Spark to infer the schema and 
    Read in JSONs from mounted S3 bucket
"""
file_location = "/mnt/0abf7f0cd605_mount/topics/0abf7f0cd605.pin/partition=0/*.json"
file_type = "json"
infer_schema = "true"
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
display(df_pin)

In [ ]:
"""
Why was this code used?

This code was used to find the data file paths in datadricks
    """
%fs ls dbfs:/mnt/0abf7f0cd605_mount/topics/

In [ ]:

file_location = "/mnt/0abf7f0cd605_mount/topics/0abf7f0cd605.geo/partition=0/*.json"
file_type = "json"
infer_schema = "true"
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
display(df_geo)

In [ ]:
file_location = "/mnt/0abf7f0cd605_mount/topics/0abf7f0cd605.user/partition=0/*.json"
file_type = "json"
infer_schema = "true"
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
display(df_user)

In [ ]:
""" why is this code used?
  
  This code is uses a function to clean the pin data into a standarised format.
  It replace misssing values with nones
  converts interger types , renames colums and reorders the dataframe
  
  
  """


from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import IntegerType

cleaned_pin_df = df_pin \
    .replace({'': None, 'User Info Error': None, 'Image src error.': None}, subset=['category', 'follower_count', 'image_src']) \
    .withColumn("follower_count", regexp_replace(regexp_replace(col("follower_count"), "k", "000"), "M", "000000")) 
cleaned_pin_df = cleaned_pin_df.withColumn("follower_count", col("follower_count").cast(IntegerType()))
cleaned_pin_df = cleaned_pin_df.withColumn("age", col("age").cast(IntegerType()))
cleaned_pin_df = cleaned_pin_df.withColumn("index", col("index").cast(IntegerType()))
cleaned_pin_df = cleaned_pin_df.withColumn("save_location", regexp_replace("save_location", "Local save in", ""))
cleaned_pin_df = cleaned_pin_df.withColumnRenamed("index","ind") 
cleaned_pin_df = cleaned_pin_df.select(["ind","unique_id","title","description","follower_count","poster_name","tag_list","is_image_or_video","image_src","save_location","category"])

display(clean ed_pin_df)


In [ ]:
""" why is this code used?
  
This code was used to clean the geo data.
It creates a new column coordinates which is a combination of 2 coloums
It Drop the latitude and longitude 
it converts to a timestamp column
It drops additional columns
It also reorders the dataframe columns
  
  """

from pyspark.sql.functions import lit,concat
cleaned_geo_df = df_user.withColumn( "coordinates",concat("latitude", lit(","), "longitude"))
cleaned_geo_df = cleaned_geo_df.drop('latitude', 'longitude')
cleaned_geo_df = cleaned_geo_df.withColumn("timestamp", to_timestamp("timestamp"))
cleaned_geo_df = cleaned_geo_df.drop('name', 'role')
cleaned_geo_df = cleaned_geo_df.select('ind','country','coordinates','timestamp')

display(cleaned_geo_df)



In [ ]:
""" why is this code used?
  
This code was used to clean the user data.
It Creates a new column user_name 
It Drops the first_name and last_name columns 
It Converts the date_joined timestamp data
It Drops additonal colums dropped 
It reorders the Data Frame
  
  """

cleaned_user_df = df_geo.withColumn("user_name",concat("first_name", lit(" "), "last_name"))
cleaned_user_df = cleaned_user_df.drop('first_name', 'last_name')
cleaned_user_df = cleaned_user_df.withColumn("date_joined", to_timestamp("date_joined"))
cleaned_user_df = cleaned_user_df.drop('role')
cleaned_user_df = cleaned_user_df.select('ind','user_name','age','date_joined')

display(cleaned_user_df)

In [ ]:
#Find the most popular Pinterest category people post to based on their country

from pyspark.sql.functions import count

most_popular_df = cleaned_geo_df.alias("geo").join(
  cleaned_pin_df.alias("pin"),
  cleaned_geo_df["ind"] == cleaned_pin_df["ind"],
  how="inner"
).groupBy("category", "country").agg(count("*").alias("category_count")).orderZBy("category_count", ascending=False)

display(most_popular_df)




In [ ]:
#Find how many posts each category had between 2018 and 2022

most_popular_cat_df = cleaned_geo_df \
    .alias("geo") \
    .join(cleaned_pin_df.alias("pin"), cleaned_geo_df["ind"] == cleaned_pin_df["ind"], how="inner") \
    .filter(year(cleaned_geo_df["timestamp"]).between(2018, 2022)) \
    .withColumn("year", year(cleaned_geo_df["timestamp"])) \
    .groupBy("category", "year") \
    .agg(count("*").alias("category_count")) \
    .orderBy("category_count", ascending=False)

display(most_popular_cat_df)



category,year,category_count
mens-fashion,2020,156
education,2021,38
christmas,2022,37
art,2021,37
finance,2021,36
diy-and-crafts,2018,36
art,2020,34
home-decor,2020,33
art,2019,32
quotes,2021,32


In [ ]:
#find the user with the most follwers in each country

most_popular_followers_df = cleaned_geo_df.alias("geo") \
    .join(cleaned_pin_df.alias("pin"), cleaned_geo_df["ind"] == cleaned_pin_df["ind"], how="inner") \
    .join(cleaned_user_df, cleaned_pin_df["ind"] == cleaned_user_df["ind"], how="inner")\
    .groupBy("country", "user_name","follower_count")\
    .agg(sum("follower_count").alias("total_follower_count"))\
    .orderBy("total_follower_count", ascending=False)

display(most_popular_followers_df)

country,user_name,follower_count,total_follower_count
Afghanistan,Alexandria Alvarado,2000000,46000000
Albania,Adam Acosta,2000000,44000000
Albania,Aaron Anderson,5000000,35000000
American Samoa,Aaron Abbott,5000000,30000000
Angola,April Brown,8000000,24000000
Algeria,Aaron Abbott,942000,22608000
Argentina,Andrew Anderson,800000,12800000
Bouvet Island (Bouvetoya),Amy Brown,5000000,10000000
Bulgaria,Andrew Alvarez,912000,9120000
Andorra,Alison Bell,1000000,9000000


In [ ]:
#find the user with the most follwers in each country

max_follower_count_df = cleaned_geo_df.alias("geo") \
    .join(cleaned_pin_df.alias("pin"), ["ind"], "inner") \
    .join(cleaned_user_df.alias("user"), ["ind"], "inner") \
    .groupBy("geo.country") \
    .agg(max("follower_count").alias("max_follower_count")) \
    .orderBy("max_follower_count", ascending=False)

display(max_follower_count_df)


country,max_follower_count
Angola,8000000
American Samoa,8000000
Azerbaijan,6000000
Burkina Faso,6000000
Bouvet Island (Bouvetoya),5000000
Albania,5000000
Algeria,5000000
United States Virgin Islands,4000000
Comoros,4000000
Guam,4000000


In [ ]:
from pyspark.sql.functions import udf, StringType, count
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# Find the most popular catgory for different age groups
most_pop_cat_df = cleaned_pin_df.alias("pin") \
    .join(cleaned_user_df.alias("user"), ["ind"], "inner")

def map_age_to_group(age):
    if age < 18:
        return 'Under 18'
    elif age < 25:
        return '18-24'
    elif age < 35:
        return '25-34'
    elif age < 45:
        return '35-44'
    else:
        return '45+'

# Define a UDF to apply the map_age_to_group function to the age column
map_age_to_group_udf = udf(map_age_to_group, StringType())

# Add a new column called age_group to the DataFrame
most_pop_cat_df = most_pop_cat_df.withColumn('age_group', map_age_to_group_udf(most_pop_cat_df['age']))

# Aggregate the data to find the count of categories for each age group
most_pop_cat_df = most_pop_cat_df.groupBy('age_group', 'category')\
    .agg(count("category")\
    .alias("category_count"))\
    .orderBy("category_count", ascending=False)


#window = Window.partitionBy("age_group").orderBy("category_count")
#most_pop_cat_df = most_pop_cat_df.withColumn("category_count", rank().over(window))

#most_pop_cat_df = most_pop_cat_df.groupBy("age_group","category") \
  #  .agg(F.max("category_count").alias("max_category_count")) \
   # .orderBy("max_category_count", ascending=False)

# Print the new DataFrame
display(most_pop_cat_df)

age_group,category,category_count
18-24,mens-fashion,152
18-24,art,86
18-24,tattoos,78
18-24,christmas,75
18-24,diy-and-crafts,68
18-24,quotes,65
18-24,education,64
18-24,finance,49
25-34,finance,46
25-34,travel,44


In [ ]:
#Find the median followers per age group

# Find the most popular catgory for different age groups
most_med_followers_per_agegroup = cleaned_pin_df.alias("pin") \
    .join(cleaned_user_df.alias("user"), ["ind"], "inner")

def map_age_to_group(age):
    if age < 18:
        return 'Under 18'
    elif age < 25:
        return '18-24'
    elif age < 35:
        return '25-34'
    elif age < 45:
        return '35-44'
    else:
        return '45+'

# Define a UDF to apply the map_age_to_group function to the age column
map_age_to_group_udf = udf(map_age_to_group, StringType())

# Add a new column called age_group to the DataFrame
most_med_followers_per_agegroup = most_med_followers_per_agegroup.withColumn('age_group', map_age_to_group_udf(most_med_followers_per_agegroup['age']))

most_med_followers_per_agegroup = most_med_followers_per_agegroup.groupBy("age_group") \
    .agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))
    
display(most_med_followers_per_agegroup)

age_group,median_follower_count
45+,3000
35-44,6000
25-34,24000
18-24,104000


In [ ]:
#Find how many users have joined between 2015 and 2020.

from pyspark.sql.functions import year, col, count

users_joined = cleaned_geo_df.alias("geo") \
    .join(cleaned_user_df.alias("user"), ["ind"], "inner") \
    .withColumn("year", year(col("user.date_joined"))) \
    .filter(year(col("user.date_joined")).between(2015, 2021)) \
    .groupBy("year") \
    .agg(count("user_name").alias("total_count")) \
    .orderBy("total_count", ascending=False)

display(users_joined)

year,total_count
2016,722
2015,704
2017,252


In [ ]:
import pandas as pd

#Find the median follower count of users have joined between 2015 and 2020.#go over
from pyspark.sql.functions import percentile_approx
users_joined = cleaned_geo_df.alias("geo") \
    .join(cleaned_user_df.alias("user"), ["ind"], "inner") \
    .withColumn("year", year(col("user.date_joined"))) \
    .filter(year(col("user.date_joined")).between(2015, 2021)) \
    .groupBy("year") \
    .agg(count("*").alias("user_name_count"))\
    .orderBy("user_name_count", ascending=False)

users_joined_df=users_joined_df("user_name_count", col("user_name_count").cast(IntegerType()))
users_joined_df = users_joined.groupBy("year") \
.agg("user_name_count".median())

# Calculate the median follower count for the entire period
users_joined_df = users_joined_df.selectExpr("percentile_approx(follower_count, 0.5) as median_follower_count")



display(users_joined)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1143679294347510> in <module> 
 11 . orderBy ( "user_name_count" , ascending = False ) 
 12 
 ---> 13 users_joined_df = users_joined_df ( "user_name_count" , col ( "user_name_count" ) . cast ( IntegerType ( ) ) ) 
 14 users_joined_df = users_joined . groupBy ( "year" ) \ 
 15 . agg ( "user_name_count" . median ( ) ) 

 TypeError : 'DataFrame' object is not callable

In [ ]:
#Find the median follower count of users have joined between 2015 and 2020.

from pyspark.sql.functions import year, count, percentile_approx

# Filter data to include only users who joined between 2015 and 2020
filtered_users_df = cleaned_geo_df.alias("geo") \
    .join(cleaned_user_df.alias("user"), ["ind"], "inner") \
    .filter(year("user.date_joined").between(2015, 2020))

# Extract year from the timestamp column
filtered_users_df = filtered_users_df.withColumn("post_year", year("geo.timestamp"))

filtered_users_df = filtered_users_df.groupBy( "post_year") \
    .agg(count("*").alias("follower_count"))

filtered_users_df = filtered_users_df.groupBy("post_year") \
    .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

# Show the result
filtered_users_df.show()



+---------+---------------------+
post_year|median_follower_count|
+---------+---------------------+
 2018| 316|
 2022| 283|
 2019| 285|
 2020| 390|
 2021| 341|
 2017| 63|
+---------+---------------------+

In [ ]:
#Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.
from pyspark.sql.functions import year, count, expr, percentile_approx, col, when

# Filter data to include only users who joined between 2015 and 2020
filtered_users_df = cleaned_geo_df.alias("geo") \
    .join(cleaned_user_df.alias("user"), ["ind"], "inner") \
    .filter(year("user.date_joined").between(2015, 2020))

# Map the age to corresponding age groups
def map_age_to_group(age):
    if age < 18:
        return 'Under 18'
    elif age < 25:
        return '18-24'
    elif age < 35:
        return '25-34'
    elif age < 45:
        return '35-44'
    else:
        return '45+'

map_age_to_group_udf = udf(map_age_to_group, StringType())
filtered_users_df = filtered_users_df.withColumn('age_group', map_age_to_group_udf(filtered_users_df['age']))

# Extract year from the timestamp column
filtered_users_df = filtered_users_df.withColumn('post_year', year("geo.timestamp"))

# Group by age group and year, calculate follower count for each group
follower_counts_df = filtered_users_df.groupBy("age_group", "post_year") \
    .agg(count("*").alias("follower_count"))

# Calculate the median follower count for each age group and year combination
median_follower_count_df = follower_counts_df.groupBy("age_group", "post_year") \
    .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

# Show the result
median_follower_count_df.show()


+---------+---------+---------------------+
age_group|post_year|median_follower_count|
+---------+---------+---------------------+
 35-44| 2021| 54|
 25-34| 2019| 87|
 45+| 2020| 46|
 18-24| 2021| 156|
 45+| 2019| 41|
 25-34| 2020| 91|
 35-44| 2022| 37|
 25-34| 2022| 73|
 25-34| 2021| 89|
 35-44| 2019| 53|
 18-24| 2019| 104|
 18-24| 2022| 138|
 18-24| 2020| 208|
 45+| 2017| 9|
 18-24| 2018| 144|
 18-24| 2017| 30|
 35-44| 2020| 45|
 45+| 2021| 42|
 25-34| 2018| 77|
 35-44| 2018| 45|
+---------+---------+---------------------+
only showing top 20 rows